In [1]:
import cenpy as c
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
%matplotlib inline

# Setup

Cenpy gives us access to a variety of US Census databases. We can see which ones via the `explorer` module:

In [2]:
dbs = c.explorer.available()
print('Number of databases: ', len(dbs))
print('First five databases: ', dbs[:5])

Number of databases:  262
First five databases:  ['SBOCSCBO12', 'ACSProfile3Y2013', '2012acs5profile', 'POPESThousing2013', 'NONEMP2001']


We're going to look at the database called `ACSSF5Y2056`. This is the American Community Survey Summary File, providing 5 year estimates from 2011-2015. The `explorer` module provides us a method to access a detailed description of the database:

In [3]:
acs_db = 'ACSSF5Y2015'
c.explorer.explain(acs_db)

{'2011-2015 American Community Survey 5-Year Estimates': 'The American Community Survey (ACS) is an ongoing survey that provides data every year -- giving communities the current information they need to plan investments and services. The ACS covers a broad range of topics about social, economic, demographic, and housing characteristics of the U.S. population.  Summary files include the following geographies: nation, all states (including DC and Puerto Rico), all metropolitan areas, all congressional districts (114th congress), all counties, all places, and all tracts and block groups.  Summary files contain the most detailed cross-tabulations, many of which are published down to block groups. The data are population and housing counts. There are over 64,000 variables in this dataset.'}

To access data from this database, we need to establish a connection to the US Census Bureau server. This might take a few seconds.

In [4]:
connection = c.base.Connection(acs_db)

The `geographies` dataframe provides the FIPS codes for each type of region defined by the Census Bureau. The first column indicates the `geoLevelId`, which is the unique identifier for a specific type of region. The `name` column describes that region, and the `requires` column describes the regions that our specific geographic ID needs before it can be specified. For example, the geo ID 150 specifies a block group, and it requires a state, county, and tract ID.

In [5]:
geographies = connection.geographies['fips']
geographies[geographies.geoLevelId == '150']

,geoLevelId,name,optionalWithWCFor,requires,wildcard
22,150,block group,tract,"[state, county, tract]",[tract]


We have a mechanism for identifying geographies. Next, we need to choose specific data that we're interested in. That's given by the `variables` dataframe:

In [6]:
variables = connection.variables
variables.head(10)

,concept,group,label,limit,predicateOnly,predicateType,required,validValues
AIANHH,Selectable Geographies,N/A,FIPS AIANHH code,0,NaN,NaN,NaN,[]
AIHHTLI,Selectable Geographies,N/A,American Indian Trust Land/Hawaiian Home Land ...,0,NaN,NaN,NaN,[]
AITS,Selectable Geographies,N/A,American Indian Tribal Subdivision (FIPS),0,NaN,NaN,NaN,[]
AITSCE,Selectable Geographies,N/A,American Indian Tribal Subdivision (Census),0,NaN,NaN,NaN,[]
ANRC,Selectable Geographies,N/A,Alaska Native Regional Corporation (FIPS),0,NaN,NaN,NaN,[]
B00001_001E,B00001. Unweighted Sample Count of the Popula...,N/A,Total,0,NaN,int,NaN,[]
B00001_001M,B00001. Unweighted Sample Count of the Popula...,N/A,Margin Of Error For!!Total,0,NaN,int,NaN,[]
B00002_001E,B00002. Unweighted Sample Housing Units,N/A,Total,0,NaN,int,NaN,[]
B00002_001M,B00002. Unweighted Sample Housing Units,N/A,Margin Of Error For!!Total,0,NaN,int,NaN,[]
B01001A_001E,B01001A. SEX BY AGE (WHITE ALONE),N/A,Total:,0,NaN,int,NaN,[]


# Explore

In [8]:
len(variables)
labels = ['SEX BY AGE',
          'MEDIAN AGE BY SEX',
          'Race',
          'B05010. Ratio of Income to Poverty Level in the Past 12 Months of Children by Living Arrangements and Nativity of Parents']

45503

In [31]:
col = 'label'
locs = variables[col].str.contains('Income|Race')
variables[['concept', col]].loc[locs]

,concept,label
B09010_002E,"B09010. Receipt of SSI, Public Assistance Inc...",Living in household with Supplemental Security...
B09010_002M,"B09010. Receipt of SSI, Public Assistance Inc...",Margin Of Error For!!Living in household with ...
B09010_003E,"B09010. Receipt of SSI, Public Assistance Inc...",Living in household with Supplemental Security...
B09010_003M,"B09010. Receipt of SSI, Public Assistance Inc...",Margin Of Error For!!Living in household with ...
B09010_004E,"B09010. Receipt of SSI, Public Assistance Inc...",Living in household with Supplemental Security...
B09010_004M,"B09010. Receipt of SSI, Public Assistance Inc...",Margin Of Error For!!Living in household with ...
B09010_005E,"B09010. Receipt of SSI, Public Assistance Inc...",Living in household with Supplemental Security...
B09010_005M,"B09010. Receipt of SSI, Public Assistance Inc...",Margin Of Error For!!Living in household with ...
B09010_006E,"B09010. Receipt of SSI, Public Assistance Inc...",Living in household with Supplemental Security...
B09010_006M,"B09010. Receipt of SSI, Public Assistance Inc...",Margin Of Error For!!Living in household with ...
